Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
    http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License. 

# MONAI Label Endoscopy App - Surgical Tool Segmentation with CVAT

***The computer-assisted intervention (CAI) workflow with MONAI Label***

In this notebook, we provide the endoscopy app example with surgical tool segmentation. The tooltracking bundle is used for the demonstration, CVAT+Nuclio as the client viewer and function deployment platform, we show how MONAI Label can provide interactive labeling support for endoscopic applications.


**Surgical Tool Segmentation with the tooltracking bundle:**

This model is trained using a flexible UNet structure with a UNet architecture decoder. Datasets use private samples from Activ Surgical.

In this tutorial, we provide publicly available sample data for demonstration. We highlight the active learning and continuous learning process with MONAI Label + Endoscopic integration.

![cvat](./figures/monailabel_endoscopy_cvat/cvat.png)


## 1. Prepare MONAI Label

### Prerequisites
- **Install MONAI Label** weekly preview release: 

In [ ]:
!pip install monailabel-weekly

### 1.1 Download endoscopy app

In [ ]:
!monailabel apps --download --name endoscopy --output apps

### 1.2 Download sample data
Several sample data are provided for this application, the sample images are from [EndoVis17](https://endovis.grand-challenge.org/) data, which is public. Users can create their own dataset format similar to the sample data. 

In [ ]:
!mkdir datasets
!wget "https://github.com/Project-MONAI/MONAILabel/releases/download/data/endoscopy_frames.zip" -O datasets/endoscopy_frames.zip
!unzip datasets/endoscopy_frames.zip -d datasets/endoscopy_frames

### 1.3 Create a validation set and modify bundle config files

Users need to provide a validation set so that the training process can evaluate model performance and save model checkpoints with the best metric. Active learning process requires the validation set.

Create the validation set following the bundle config files, the path and filenames need to be consistent between file system and the train.json.

Modify the train.json in the bundle config file, the ```dataset_dir```, ```val_images```, and ```val_labels``` are required to be the same with file system.

![data1](./figures/monailabel_endoscopy_cvat/data1.png)

![bundle](./figures/monailabel_endoscopy_cvat/bundle_config.png)



## 2.  Setup CVAT and Nuclio


CVAT and Nuclio are not maintained by MONAI Label team.  We provide minimized setup steps for using CVAT, more details about CVAT preparation can be found in:

- [CVAT Viewer Configuration](https://opencv.github.io/cvat/docs/administration/basics/installation/)
- [Semi-automatic and Automatic Annotation](https://opencv.github.io/cvat/docs/administration/advanced/installation_automatic_annotation/)

For following step, setup in the local machine (e.g., Ubuntu), and execute commands.


### 2.1 Step 1: Get CVAT from GitHub and checkout to version 2.1.0

Open a **Terminal** in local machine and execute:

```bash
git clone https://github.com/opencv/cvat
cd cvat
# Checkout to required stable version to v2.1.0
git checkout v2.1.0
```

### 2.2 Step 2: Set the Host and CVAT version, configure CVAT


```bash
# use real-ip instead of localhost to make the CVAT projects sharable
export CVAT_HOST=127.0.0.1
export CVAT_VERSION=v2.1.0
# Start CVAT from docker-compose, make sure the IP and port are available.
python3 -m pip install setuptools docker-compose # this is optional if docker-compose is already installed.
docker-compose -f docker-compose.yml -f components/serverless/docker-compose.serverless.yml up -d
# Create CVAT username and password
docker exec -it cvat bash -ic 'python3 ~/manage.py createsuperuser'
```

The setup process takes port 8070, 8080, 8090, if alternative ports are preferred, refer to [CVAT Guide](https://opencv.github.io/cvat/docs/administration/basics/installation/)

CVAT should be on after this step, refer to **http://127.0.0.1:8080** in Chrome to open CVAT, login with the superuser username and password:

![cvat1](./figures/monailabel_endoscopy_cvat/cvat1.png)


### 2.3 Step 3: Setup Nuclio container platform for function container management

In the local host machine, start a **Terminal** to open a command line interface.

```bash
# Get Nuclio dashboard 
wget https://github.com/nuclio/nuclio/releases/download/1.5.16/nuctl-1.5.16-linux-amd64
chmod +x nuctl-1.5.16-linux-amd64
ln -sf $(pwd)/nuctl-1.5.16-linux-amd64 /usr/local/bin/nuctl

```

### 2.4 Step 4: Endoscopic Models Deployment with MONAI Label

This step is to deploy MONAI Label plugin with endoscopic models using Nuclio tool.

```bash
# make a working folder in local machine, name workspace
mkdir workspace
# Get CVAT plugin folder from the Github
git clone https://github.com/Project-MONAI/MONAILabel.git
# Deploy all endoscopy models
./MONAILabel/plugins/cvat/deploy.sh endoscopy
# Or to deploy specific function and model, e.g., tooltracking
./MONAILabel/plugins/cvat/deploy.sh endoscopy tooltracking
```

After model deployment, users can see the model names in the **Models** tag of CVAT

![cvat2](./figures/monailabel_endoscopy_cvat/cvat2.png)

To check or monitor the status of deployed function container, users can open Nuclio platform at http://127.0.0.1:8070 by default and see whether deployed models are running.

![cvat3](./figures/monailabel_endoscopy_cvat/cvat3.png)



## 3. Usecase 1: Inference with CVAT+MONAI Label Function 

MONAI Label deployed models can be directly used in the CVAT without a monailabel start_server command, as the deployed functions are running inside the containers. Users can check the status in the deployed function container by logging docker image, e.g, tooltracking container: ```docker container logs nuclio-nuclio-monailabel.endoscopy.tooltracking``` in JupyterLab terminal.

In the following steps, users can download same sample data by:

```bash
mkdir datasets
wget "https://github.com/Project-MONAI/MONAILabel/releases/download/data/endoscopy_frames.zip" -O datasets/endoscopy_frames.zip
unzip datasets/endoscopy_frames.zip -d datasets/endoscopy_frames
```

Select frames to upload to CVAT.

### 3.1 Create task and upload images to CVAT with UI

Click the "+" icon in the **Tasks** tag, create a name and add a label, then click "Done". Select and upload an image. Then click **Submit** to create a task.

![cvat4](./figures/monailabel_endoscopy_cvat/cvat4.png)

A "job" will be shown in the CVAT, click the "job" to open the labeling task:

![cvat5](./figures/monailabel_endoscopy_cvat/cvat5.png)


### 3.2 Inference with MONAI Label model

On the labeling task page, click the **AI Tools** on the left panel. Select **Detectors**, choose **tooltracking**
in the dropdown scroll. Pick label and finally click **Annotate** button. 

![cvat6](./figures/monailabel_endoscopy_cvat/cvat6.png)

The predicted mask will be shown as polygon in the CVAT, users can edit the mask and click **Save** icon on the top. Once finished annotation, click "Menu", change the job state to "complete", and "Finish the job".  

![cvat7](./figures/monailabel_endoscopy_cvat/cvat7.png)

Users can export annoated masks using CVAT UI.

## 4. Usecase 2: Active Learning Workflow with MONAI Label + CVAT

The advanced feature of the endoscopic application is to actively select and fine-tune models. 

Below is a step-by-step tutorial on using MONAI Label+CVAT for auto segmentation, active learning and fine-tuning models using surgical tool segmentation bundle. Pre-trained model is provided by the bundle and loaded by MONAI Label server.

### 4.1 Start MONAI Label server for Active Learning

Connect the MONAI Label server with cvat datastore and URL, set USERNAME and PASSWORD same as login credentials for CVAT.

```bash
# cd to the folder where have the downloaded apps and data
cd /opt/tutorials/monaiLabel

export MONAI_LABEL_DATASTORE=cvat
export MONAI_LABEL_DATASTORE_URL=http://127.0.0.1:8080
# the USERNAME and PASSWARD are the authentications created with command:
# "docker exec -it cvat bash -ic 'python3 ~/manage.py createsuperuser"
export MONAI_LABEL_DATASTORE_USERNAME=<USERNAME>
export MONAI_LABEL_DATASTORE_PASSWORD=<PASSWORD>

# Specify the path of the downloaded endoscopy app (--app <APPPATH>) and the sample dataset (--studies <DATAPATH>)
monailabel start_server \
  --app apps/endoscopy \
  --studies datasets/endoscopy_frames \
  --conf models tooltracking \
  --conf epistemic_enabled true \
  --conf epistemic_top_k 2
```

Then MONAI Label server will start scoring inference for each unlabeled image using epistemic scoring method. Users can set the number of image to label in CVAT for each iteration, the ```--conf epistemic_top_k 10``` specifies the workload for each active learning loop.

MONAI Label server will automatically upload selected images to CVAT.
Note: if images have been uploaded or the prior iteration's task is not complete, MONAI Label will prevent uploading more.

The automatically created project and tasks can be seen here:
![cvat8](./figures/monailabel_endoscopy_cvat/cvat8.png)

![cvat9](./figures/monailabel_endoscopy_cvat/cvat9.png)

### 4.2 Labeling with CVAT and MONAI Label AI Tool

The created tasks will run inference automatically, users can see predicted mask in the job page, edit them and follow same annotating process in Section 3. Once finished, click save, change job state and finish the job.

![cvat10](./figures/monailabel_endoscopy_cvat/cvat10.png)

### 4.3 Trigger Model Fine-Tuning.

Continue to annotate until all created jobs are completed. MONAI Label will monitor the datastore status to check whether all jobs are in "completed" status.

Once all jobs are completed, MONAI Label server will automatically trigger model fine-tuning and train the model with newly generated labels.

![cvat11](./figures/monailabel_endoscopy_cvat/cvat11.png)


### 4.4 Scoring and Create Next iteration for CVAT

Users will get a fine-tuned model if users provided the validation set.

![cvat12](./figures/monailabel_endoscopy_cvat/cvat12.png)

## 5. Conclusion

This tutorial demonstrates MONAI Label usage with endoscopy app, MONAI Label provides an advanced active learning workflow for endoscopic applications. Best with CVAT and Nuclio deployment, the MONAI Label has the leading functionality of computer assisted intervention tasks. 